__собираем данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

loc_file_path = 'data/location.pkl' # таблица адресов
result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx']

In [7]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd
        .read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

17454


In [8]:
assert len(data_avito)>0

In [9]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17454 entries, 0 to 17453
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       17454 non-null  int64         
 1   title          17454 non-null  object        
 2   price          17454 non-null  int64         
 3   adr            17454 non-null  object        
 4   description    17454 non-null  object        
 5   obj_name       17454 non-null  object        
 6   ts             17454 non-null  datetime64[ns]
 7   avito_page     17454 non-null  int64         
 8   nrooms         17454 non-null  int64         
 9   floor          17454 non-null  int64         
 10  nfloors        17454 non-null  int64         
 11  area           17454 non-null  float64       
 12  is_studio      17454 non-null  bool          
 13  is_apartment   17454 non-null  bool          
 14  is_part        17454 non-null  bool          
 15  is_auction     1745

In [10]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [11]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl']

In [12]:
data_cian = pd.concat([
        # pd.read_excel(f)
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [13]:
data_cian.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009 entries, 0 to 3008
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     3009 non-null   object        
 1   OfferSubtitle  3009 non-null   object        
 2   Deadline       3009 non-null   object        
 3   MainPrice      3009 non-null   object        
 4   PriceInfo      3009 non-null   object        
 5   GeoLabel       3009 non-null   object        
 6   TimeLabel      3009 non-null   object        
 7   LinkArea       3009 non-null   object        
 8   Description    3009 non-null   object        
 9   cian_page      3009 non-null   int64         
 10  ts             3009 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 258.7+ KB


In [14]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [15]:
# data_cian

In [16]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

3009


In [17]:
assert len(data_cian)>0

In [18]:
data_cian.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009 entries, 0 to 3008
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          3009 non-null   object        
 1   obj_name       3009 non-null   object        
 2   adr            3009 non-null   object        
 3   nrooms         3009 non-null   int64         
 4   floor          3009 non-null   int64         
 5   nfloors        3009 non-null   int64         
 6   area           3009 non-null   float64       
 7   is_studio      3009 non-null   bool          
 8   is_apartment   3009 non-null   bool          
 9   is_part        3009 non-null   bool          
 10  is_auction     3009 non-null   bool          
 11  is_openspace   3009 non-null   bool          
 12  is_roof        3009 non-null   bool          
 13  is_SNT         3009 non-null   bool          
 14  price          3009 non-null   int64         
 15  priceM         3009 n

In [19]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [20]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

20463


### обновляем таблицу адресов

In [21]:
from lib.locator import LocationUpdater
from lib.locator import AddressTransformerSev

loc = LocationUpdater(
        address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-12 14:16:14 | LocationUpdater: 3149 addresses in location table
[INFO    ] 2022-09-12 14:16:14 | LocationUpdater: 3281 addresses total
[INFO    ] 2022-09-12 14:16:14 | LocationUpdater: 3149 addresses defined
[INFO    ] 2022-09-12 14:16:14 | LocationUpdater: 132 addresses undefined


  0%|          | 0/132 [00:00<?, ?it/s]

[INFO    ] 2022-09-12 14:18:49 | LocationUpdater: 89 new addresses found


In [22]:
loc.sample(3)

,adr,latitude,longitude,truncated
737,"улица Вакуленчука, 5",44.59,33.48,False
2389,"Севастополь, район Ленинский, микрорайон Острякова, улица Николая Музыки, 36",44.56,33.53,True
2113,"Севастополь, район Балаклавский, шоссе Фиолентовское, 134 к4",44.51,33.49,False


In [23]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [24]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 20463
записей без геометки: 988


In [25]:
assert len(data)>0
data.to_pickle(result_file_path)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20463 entries, 0 to 20462
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       17454 non-null  float64       
 1   title          20463 non-null  object        
 2   price          20463 non-null  int64         
 3   adr            20463 non-null  object        
 4   description    20463 non-null  object        
 5   obj_name       20463 non-null  object        
 6   ts             20463 non-null  datetime64[ns]
 7   avito_page     17454 non-null  float64       
 8   nrooms         20463 non-null  int64         
 9   floor          20463 non-null  int64         
 10  nfloors        20463 non-null  int64         
 11  area           20463 non-null  float64       
 12  is_studio      20463 non-null  bool          
 13  is_apartment   20463 non-null  bool          
 14  is_part        20463 non-null  bool          
 15  is_auction     2046

In [27]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)